In [39]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import geojson
import geopandas as gpd
from bs4 import BeautifulSoup 
import io
import zipfile
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

### Primeros datos generados para la propuesta

In [2]:
url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Chile_2010'
respuesta = requests.get(url).text

soup = BeautifulSoup(respuesta)

In [25]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[3]
equipos = informacion_posiciones.find_all('tr')[1::]

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(10)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipo','pts','j','g','e','p','gf','gc','dif'])
df.set_index(['POS'])

,Equipo,pts,j,g,e,p,gf,gc,dif
POS,,,,,,,,,
1.,Universidad Católica,74,34,23,5,6,77,39,+38
2.,Colo-Colo 1,71,34,22,5,7,67,34,+33
3.,Audax Italiano,65,34,20,5,9,75,58,+17
4.,Universidad de Chile,64,34,20,4,10,75,48,+27
5.,Unión Española,52,34,14,10,10,58,50,+8
6.,Huachipato,48,34,12,12,10,44,40,+4
7.,Deportes La Serena,45,34,13,6,15,45,59,-14
8.,Santiago Wanderers,45,34,12,9,13,48,52,-4
9.,Unión San Felipe,43,34,12,7,15,38,47,-9


In [59]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup.find('div',{'class':'mw-parser-output'}).find('table',border="0").find_all('tr')[1::]

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(0)
tabla_equipos.pop(19)
tabla_equipos.pop(18)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    data.pop(1)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Ciudad', 'Estadio', 'Capacidad Estadio'])
datos_equipos_df

,Equipo,Entrenador,Ciudad,Estadio,Capacidad Estadio
0,Audax Italiano,Omar Labruna,Santiago,Bicentenario de La Florida,12.000
1,Cobreloa,Mario Soto,Calama,Municipal de Calama,13.000
2,Cobresal,Luis Musrri,El Salvador,El Cobre,15.000
3,Colo Colo,Diego Cagna,Santiago,Monumental David Arellano,47.017
4,Deportes La Serena,Víctor Hugo Castañeda,La Serena,La Portada,14.000
5,Everton,Diego Osella (interino),Viña del Mar,Sausalito,18.037
6,Huachipato,Arturo Salah,Talcahuano,CAP,10.500
7,Ñublense,Luis Marcoleta,Chillán,Bicentenario Nelson Oyarzún Arenas,12.000
8,O'Higgins,Marco Antonio Figueroa,Rancagua,El Teniente,14.450
9,Palestino,Gustavo Benítez,Santiago,Municipal de La Cisterna,12.000


### Datos apertura 2015

In [73]:
url_apertura_2015 = 'https://es.wikipedia.org/wiki/Torneo_Apertura_2015_(Chile)'
respuesta = requests.get(url_apertura_2015).text

soup_apertura_2015 = BeautifulSoup(respuesta)

In [96]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_apertura_2015.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[3].find_all('tr')[1::]
len(tabla_equipos)

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(18)
tabla_equipos.pop(17)
tabla_equipos.pop(16)
tabla_equipos.pop(0)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(7):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Ciudad', 'Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Ciudad,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Jorge Pellicer,Santiago (La Florida),Bicentenario de La Florida,12.000,Dalponte,Ideal
1,Cobresal,Rubén Vallejos,El Salvador,El Cobre,20.752,Lotto,PF
2,Colo-Colo,José Luis Sierra,Santiago (Macul),Monumental David Arellano,47.141,Under Armour,DirecTV
3,Deportes Antofagasta,Beñat San José,Antofagasta,Calvo y Bascuñán,21.178,Cafu,Escondida
4,Deportes Iquique,Jaime Vera,Iquique,Tierra de Campeones,12.000,Lotto,UNAP
5,Huachipato,Hugo Vilches,Talcahuano,CAP,10.579,Mitre,
6,O'Higgins,Pablo Sánchez,Rancagua,El Teniente,15.600,New Balance,VTR
7,Palestino,Pablo Guede,Santiago (La Cisterna),Municipal de La Cisterna,12.000,Training,Bank Of Palestine
8,San Luis de Quillota,Miguel Ramírez,Quillota,Lucio Fariña,7.703,Dalponte,CVU Transportes
9,San Marcos de Arica,Marco Antonio Figueroa,Arica,Carlos Dittborn,10.000,Dalponte,TPA


In [108]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_apertura_2015.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[6]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(12)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['Premio','POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF','REND'])
df = df.drop(['Premio'], axis=1)
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF,REND
POS,,,,,,,,,,
1.,Colo-Colo (C),33,15,11,0,4,24,14,+10,"73,3%"
2.,Universidad Católica,32,15,10,2,3,33,15,+18,"71,1%"
3.,Universidad de Concepción,28,15,9,1,5,29,19,+10,"62,2%"
4.,Palestino,28,15,8,4,3,23,17,+6,"62,2%"
5.,Audax Italiano,26,15,7,5,3,22,19,+3,"57,8%"
6.,Unión Española,24,15,7,3,5,30,20,+10,"53,3%"
7.,O'Higgins,23,15,7,2,6,23,23,0,"51,1%"
8.,Santiago Wanderers,20,15,5,5,5,20,19,+1,"44,4%"
9.,Cobresal,18,15,6,0,9,23,28,-5,40%


### Datos clausura 2015

In [109]:
url_clausura_2015 = 'https://es.wikipedia.org/wiki/Torneo_Clausura_2015_(Chile)'
respuesta = requests.get(url_clausura_2015).text

soup_clausura_2015 = BeautifulSoup(respuesta)

In [128]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_clausura_2015.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[0].find_all('tr')[1::]
len(tabla_equipos)

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(18)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(7):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Ciudad', 'Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Ciudad,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Jorge Pellicer,Santiago (La Florida),Municipal de La Florida,12.000,Dalponte,Ideal
1,Barnechea,Francisco Bozán,Santiago (Lo Barnechea),San Carlos de Apoquindo[1]​,20.000,Mitre,Red MTS
2,Cobreloa,Marco Antonio Figueroa,Calama,Zorros del Desierto,12.000,Lotto,Finning CAT
3,Cobresal,Dalcio Giovagnoli,El Salvador,El Cobre,20.752,Lotto,PF
4,Colo-Colo,Héctor Tapia,Santiago (Macul),Monumental David Arellano,47.017,Under Armour,DirecTV
5,Deportes Antofagasta,José Cantillana,Antofagasta,Calvo y Bascuñán,21.178,Uhlsport,Escondida
6,Deportes Iquique,Nelson Acosta,Iquique,Tierra de Campeones,12.000,Lotto,Sopranil
7,Huachipato,Hugo Vilches,Talcahuano,CAP,10.022,Mitre,
8,Ñublense,Fernando Díaz,Chillán,Nelson Oyarzún Arenas,12.000,Cafú,PF
9,O'Higgins,Pablo Sánchez,Rancagua,El Teniente,15.600,New Balance,VTR


In [133]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_clausura_2015.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[3]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(12)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['Premio','POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF','REND'])
df = df.drop(['Premio'], axis=1)
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF,REND
POS,,,,,,,,,,
1.,Cobresal (C),34,17,10,4,3,29,20,+9,66.7%
2.,Colo Colo,32,17,10,2,5,32,21,+11,62.7%
3.,Huachipato,31,17,9,4,4,29,28,+1,60.8%
4.,Universidad Católica,29,17,8,5,4,40,31,+9,56.9%
5.,Unión La Calera,27,17,8,3,6,33,26,+7,52.9%
6.,Universidad de Concepción,27,17,8,3,6,27,26,+1,52.9%
7.,Universidad de Chile,26,17,8,2,7,36,27,+9,51%
8.,San Marcos de Arica,24,17,6,6,5,26,19,+7,47.1%
9.,O'Higgins,24,17,6,6,5,21,23,-2,47.1%


### Datos Apertura 2016

In [142]:
url_apertura_2016 = 'https://es.wikipedia.org/wiki/Torneo_Apertura_2016_(Chile)'
respuesta = requests.get(url_apertura_2016).text

soup_apertura_2016 = BeautifulSoup(respuesta)

In [147]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_apertura_2016.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[4].find_all('tr')[1::]
len(tabla_equipos)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Hugo Vilches,Bicentenario de La Florida,12.000,Macron,Traverso
1,Cobresal[1]​,Dalcio Giovagnoli,El Cobre,20.000,Lotto,PF
2,Colo-Colo,Pablo Guede,Monumental David Arellano,47.000,Under Armour,DirecTV
3,Deportes Antofagasta,Fernando Vergara,Regional Calvo y Bascuñán,21.000,Cafu,Escondida
4,Deportes Iquique,Jaime Vera,Tierra de Campeones[2]​,12.000,Rete,UNAP
5,Deportes Temuco,Luis Landeros,Bicentenario Germán Becker,18.500,Joma,Rosen
6,Everton,Pablo Sánchez,Sausalito,23.500,Pirma,Viña del Mar
7,Huachipato,Miguel Ponce,Huachipato-CAP Acero,10.500,Mitre,
8,O'Higgins,Cristián Arán,El Teniente,15.500,New Balance,VTR
9,Palestino,Nicolás Córdova,Municipal de La Cisterna[3]​,10.000,Training,Bank Of Palestine


In [152]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_apertura_2016.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[6]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(12)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['Premio','POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF','REND'])
df

df = df.drop(['Premio'], axis=1)
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF,REND
POS,,,,,,,,,,
1.,Universidad Católica,31,15,9,4,2,37,18,+19,"68,8%"
2.,Deportes Iquique,28,15,8,4,3,28,22,+6,"62,2%"
3.,Unión Española,27,15,8,3,4,27,20,+7,60%
4.,O'Higgins,26,15,7,5,3,23,15,+8,"57,8%"
5.,Colo-Colo,23,15,6,5,4,26,20,+6,"51,1%"
6.,Palestino,21,15,6,3,6,25,19,+6,"46,7%"
7.,Universidad de Chile,21,15,5,6,4,23,22,+1,"46,7%"
8.,San Luis,21,15,6,3,6,21,22,-1,"46,7%"
9.,Deportes Antofagasta,19,15,5,4,6,17,22,-5,"42,2%"


### Datos Clausura 2016

In [153]:
url_clausura_2016 = 'https://es.wikipedia.org/wiki/Torneo_Clausura_2016_(Chile)'
respuesta = requests.get(url_clausura_2016).text

soup_clausura_2016 = BeautifulSoup(respuesta)

In [166]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_clausura_2016.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[1].find_all('tr')[1::]
len(tabla_equipos)

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(17)
tabla_equipos.pop(0)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(7):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Ciudad','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Ciudad,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Jorge Pellicer,Santiago (La Florida),Bicentenario de La Florida,12.000,KS7,Traverso
1,Cobresal,Dalcio Giovagnoli,El Salvador,El Cobre,20.752,Lotto,PF
2,Colo-Colo,José Luis Sierra,Santiago (Macul),Monumental David Arellano,45.347,Under Armour,DirecTV
3,Deportes Antofagasta,Beñat San José,Antofagasta,Calvo y Bascuñán,21.178,Cafu,Escondida
4,Deportes Iquique,Jaime Vera Rodríguez,Iquique,Tierra de Campeones,12.000,Lotto,UNAP
5,Huachipato,Miguel Ponce,Talcahuano,Huachipato-CAP Acero,10.022,Mitre,
6,O'Higgins,Cristián Arán,Rancagua,El Teniente,15.600,New Balance,VTR
7,Palestino,Nicolás Córdova,Santiago (La Cisterna),Municipal de La Cisterna,12.000,Training,Bank of Palestine
8,San Luis de Quillota,Miguel Ramírez,Quillota,Lucio Fariña Fernández,7.703,Dalponte,PF
9,San Marcos de Arica,Emiliano Astorga,Arica,Carlos Dittborn,10.000,Walon,TPA


In [169]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_clausura_2016.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[4]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(12)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['Premio','POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF','REND'])
df

df = df.drop(['Premio'], axis=1)
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF,REND
POS,,,,,,,,,,
1.,Universidad Católica (C),29,15,9,2,4,33,25,+8,64.4%
2.,Colo-Colo,28,15,8,4,3,19,11,+8,62.2%
3.,O'Higgins,28,15,8,4,3,28,24,+4,62.2%
4.,Palestino,25,15,6,7,2,24,18,+6,55.6%
5.,Universidad de Concepción,25,15,8,1,6,22,24,-2,55.6%
6.,Santiago Wanderers,23,15,6,5,4,31,26,+5,51.1%
7.,Deportes Antofagasta,21,15,6,3,6,23,18,+5,46.7%
8.,Deportes Iquique,19,15,4,7,4,23,22,+1,42.2%
9.,Huachipato,19,15,4,7,4,26,27,-1,42.2%


### Datos apertura 2017

In [170]:
url_clausura_2017 = 'https://es.wikipedia.org/wiki/Torneo_Clausura_2017_(Chile)'
respuesta = requests.get(url_clausura_2017).text

soup_clausura_2017 = BeautifulSoup(respuesta)

In [184]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_clausura_2017.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[2].find_all('tr')[1::]
len(tabla_equipos)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Hugo Vilches,Bicentenario de La Florida,12.000,Macron,Traverso
1,Cobresal,Rubén Vallejos,El Cobre,20.000,Lotto,PF
2,Colo-Colo,Pablo Guede,Monumental David Arellano,47.000,Under Armour,DirecTV
3,Deportes Antofagasta,Fernando Vergara,Regional Calvo y Bascuñán,21.000,Cafu,Escondida
4,Deportes Iquique,Jaime Vera,Tierra de Campeones[1]​,12.000,Rete,UNAP
5,Deportes Temuco,Dalcio Giovagnoli,Bicentenario Germán Becker,18.500,Joma,Rosen
6,Everton,Pablo Sánchez,Sausalito,23.500,Pirma,Viña del Mar
7,Huachipato,Miguel Ponce,Huachipato-CAP Acero,10.500,Mitre,PF
8,O'Higgins,Gabriel Milito,El Teniente,15.500,New Balance,VTR
9,Palestino,German Cavalieri,Municipal de La Cisterna,10.000,Training,Bank Of Palestine


In [182]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_clausura_2017.find('div', class_='mw-parser-output').find_all('table',{'class':'wikitable'})[4]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(11)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF','REND'])
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF,REND
POS,,,,,,,,,,
1.,Universidad de Chile (C),30,15,9,3,3,24,11,+13,"66,6%"
2.,Colo-Colo,29,15,8,5,2,29,13,+16,"64,4%"
3.,Universidad de Concepción,24,15,6,6,3,16,13,+3,"53,3%"
4.,Universidad Católica,23,15,7,2,6,25,23,+2,"51,1%"
5.,Unión Española,22,15,6,4,5,25,21,+4,"48,9%"
6.,Deportes Iquique,22,15,5,7,3,23,20,+3,"48,9%"
7.,Audax Italiano,22,15,6,4,5,23,23,0,"48,9%"
8.,O'Higgins,22,15,6,4,5,20,20,0,"48,9%"
9.,Deportes Temuco,22,15,6,4,5,17,17,0,"48,9%"


### Datos transición 2017

In [185]:
url_transicion_2017 = 'https://es.wikipedia.org/wiki/Torneo_de_Transici%C3%B3n_2017_(Chile)'
respuesta = requests.get(url_transicion_2017).text

soup_transicion_2017 = BeautifulSoup(respuesta)

In [187]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_transicion_2017.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[4].find_all('tr')[1::]
len(tabla_equipos)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Hugo Vilches,Bicentenario de La Florida,12.000,Macron,Traverso
1,Colo-Colo,Pablo Guede,Monumental David Arellano,47.000,Under Armour,DirecTV
2,Curicó Unido,Luis Marcoleta,La Granja,8.278,Onefit,Multihogar
3,Deportes Antofagasta,Nicolás Larcamón,Bicentenario Calvo y Bascuñán,21.000,Cafu,Escondida
4,Deportes Iquique,Erick Guerrero,Municipal de Cavancha[2]​,5.500,Rete,UNAP
5,Deportes Temuco,Dalcio Giovagnoli,Bicentenario Germán Becker,18.500,Joma,Rosen
6,Everton,Pablo Sánchez,Sausalito,23.000,Pirma,Fox Sports Chile
7,Huachipato,César Vigevani,Huachipato-CAP Acero,10.500,Mitre,PF
8,O'Higgins,Gabriel Milito,El Teniente,15.000,New Balance,VTR
9,Palestino,German Cavalieri,Municipal de La Cisterna,10.000,Training,Bank Of Palestine


In [220]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_transicion_2017.find('div', class_='mw-parser-output').find('table',{'align':'center'})
equipos = informacion_posiciones.find_all('tr')[1::]
len(equipos)

# Se eliminan los indices que no tienen relación a la información buscada
equipos.pop(16)

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(10)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF'])
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF
POS,,,,,,,,,
1.,Colo-Colo[n. 1]​,33,15,10,3,2,33,13,20
2.,Unión Española,31,15,9,4,2,18,12,6
3.,Universidad de Chile[n. 2]​,30,15,9,3,3,21,18,3
4.,Everton,26,15,7,5,3,24,19,5
5.,Audax Italiano,25,15,7,4,4,24,19,5
6.,Deportes Temuco,22,15,5,7,3,15,12,3
7.,Deportes Antofagasta,21,15,5,6,4,14,13,1
8.,Curicó Unido,18,15,5,3,7,11,14,-3
9.,San Luis,18,15,5,3,7,15,20,-5


### Datos campeonato largo 2018

In [221]:
url_campeonato_2018 = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Chile_2018'
respuesta = requests.get(url_campeonato_2018).text

soup_campeonato_2018 = BeautifulSoup(respuesta)

In [225]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_campeonato_2018.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[4].find_all('tr')[1::]
len(tabla_equipos)

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(17)
tabla_equipos.pop(16)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Juan José Ribera,Bicentenario de La Florida,12 000[4]​,Macron,Traverso
1,Colo-Colo,Héctor Tapia,Monumental David Arellano,47 347[5]​,Under Armour,MG Motor
2,Curicó Unido,Jaime Vera,La Granja,8278[6]​,One Fit,Multihogar
3,Deportes Antofagasta,Gerardo Ameli,Bicentenario Calvo y Bascuñán,21 000[7]​,CAFU - Tro Deportes,Minera Escondida
4,Deportes Iquique,Luis Musrri,Municipal de Cavancha [in. 1]​,5500[8]​,Rete,Universidad Arturo Prat
5,Deportes Temuco,Miguel Ponce,Bicentenario Germán Becker,18 000[9]​,Capelli Sport,Rosen
6,Everton,Javier Torrente,Sausalito,21 000[10]​,Pirma,Fox Sports
7,Huachipato,Nicolás Larcamón,Huachipato-CAP Acero,10 500[11]​,Mitre,PF Alimentos
8,O'Higgins,Marco Antonio Figueroa,El Teniente,14 000[12]​,Adidas,VTR
9,Palestino,Ivo Basay,Municipal de La Cisterna,12 000[13]​,Training Professional,Bank of Palestine


In [226]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_campeonato_2018.find('div', class_='mw-parser-output').find('table',{'align':'center'})
equipos = informacion_posiciones.find_all('tr')[1::]
len(equipos)

# Se eliminan los indices que no tienen relación a la información buscada
equipos.pop(16)

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(10)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF'])
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF
POS,,,,,,,,,
1.,Universidad Católica,61,30,17,10,3,39,25,+14
2.,Universidad de Concepción,58,30,18,4,8,45,31,+14
3.,Universidad de Chile,57,30,18,3,9,45,37,+8
4.,Deportes Antofagasta,53,30,14,11,5,48,33,+15
5.,Colo-Colo,43,30,12,7,11,40,38,+2
6.,Unión La Calera,43,30,13,4,13,38,38,0
7.,Unión Española,41,30,9,14,7,44,42,+2
8.,O'Higgins,41,30,12,5,13,41,41,0
9.,Huachipato,39,30,10,9,11,39,32,+7


### Datos campeonato largo 2019

In [227]:
url_campeonato_2019 = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Chile_2019'
respuesta = requests.get(url_campeonato_2019).text

soup_campeonato_2019 = BeautifulSoup(respuesta)

In [235]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_campeonato_2019.find('div',{'class':'mw-parser-output'}).find_all('table',class_='wikitable')[3].find_all('tr')[1::]
len(tabla_equipos)

# Se eliminan los indices que no tienen relación a la información buscada
tabla_equipos.pop(16)


# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Juan José Ribera,Bicentenario de La Florida,12.000,Macron,Traverso
1,Cobresal,Gustavo Huerta,El Cobre,12.000,KS7,PF Alimentos
2,Colo-Colo,Mario Salas,Monumental David Arellano,47.347,Umbro,MG Motor
3,Coquimbo Unido,Patricio Graff,Francisco Sánchez Rumoroso,18.000,CAFU,PF Alimentos
4,Curicó Unido,Hugo Vilches,La Granja,8.278,OneFit,Multihogar
5,Deportes Antofagasta,Juan Manuel Azconzábal,Bicentenario Calvo y Bascuñán,21.100,CAFU,Minera Escondida
6,Deportes Iquique,Jaime Vera,Municipal de Cavancha,5.500,Rete,Universidad Arturo Prat
7,Everton,Javier Torrente,Sausalito,21.000,Pirma,Marathonbet
8,Huachipato,Gustavo Florentín,Huachipato - CAP Acero,10.500,OneFit,PF Alimentos
9,O'Higgins,Marco Antonio Figueroa,El Teniente,14.000,Adidas,Sun Monticello


In [242]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_campeonato_2019.find('div', class_='mw-parser-output').find_all('table',{'align':'center'})[1]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# Se eliminan los indices que no tienen relación a la información buscada
equipos.pop(16)

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(10)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF'])
df.set_index('POS')

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF
POS,,,,,,,,,
1.,Universidad Católica,53,24,16,5,3,44,14,+30
2.,Colo-Colo,40,24,11,7,6,37,30,+7
3.,Palestino,38,24,10,8,6,42,31,+11
4.,Unión La Calera,37,25,9,10,6,29,23,+6
5.,Coquimbo Unido,34,24,8,10,6,29,27,+2
6.,Huachipato,34,24,9,7,8,31,30,+1
7.,Audax Italiano,34,24,10,4,10,35,35,0
8.,O'Higgins,34,24,10,4,10,34,35,-1
9.,Unión Española,34,25,10,4,11,32,35,-3


### Datos campeonato largo 2020

In [243]:
url_campeonato_2020 = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Chile_2020'
respuesta = requests.get(url_campeonato_2020).text

soup_campeonato_2020 = BeautifulSoup(respuesta)

In [249]:
# Se encuentra la tabla de datos de los equipos mediante el metodo find y find_all de bs4
tabla_equipos = soup_campeonato_2020.find('div',{'class':'mw-parser-output'}).find_all('table',class_='striped')[0].find_all('tr')[1::]
len(tabla_equipos)

# Se genera el df con los datos de los equipos
datosequipo = []
for equipos in tabla_equipos:
    data = []
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador','Estadio', 'Capacidad Estadio', 'Marca', 'Patrocinador'])
datos_equipos_df

,Equipo,Entrenador,Estadio,Capacidad Estadio,Marca,Patrocinador
0,Audax Italiano,Pablo Sánchez,Bicentenario de La Florida,12.000,Macron,Traverso
1,Cobresal,Gustavo Huerta,El Cobre,12.000,KS7,PF Alimentos
2,Colo-Colo,Gustavo Quinteros,Monumental,47.347,Umbro,MG Motor
3,Coquimbo Unido,Juan Ribera,Francisco Sánchez,18.750,Cafú,PF Alimentos
4,Curicó Unido,Martín Palermo,La Granja,8.278,OneFit,Multihogar
5,Deportes Antofagasta,Héctor Tapia,Calvo y Bascuñán,21.100,Cafú,Minera Escondida
6,Deportes Iquique,Cristián Leiva,Tierra de Campeones,13.171,Rete,UNAP
7,Deportes La Serena,Miguel Ponce,La Portada,18.000,OneFit,
8,Everton,Roberto Sensini,Sausalito,22.000,Charly,Claro
9,Huachipato,Juan Luvera,Huachipato - CAP Acero,10.032,OneFit,PF Alimentos


In [256]:
# Se encuentra la tabla de posiciones de los equipos mediante el metodo find y find_all de bs4
informacion_posiciones = soup_campeonato_2020.find('div', class_='mw-parser-output').find_all('table',{'align':'center'})[1]
equipos = informacion_posiciones.find_all('tr')[1::]
equipos

# # Se eliminan los indices que no tienen relación a la información buscada
# equipos.pop(16)

# Se genera el df de las posiciones de los equipos
aux = []
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(9)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['Equipos','PTS','PJ','PG','PE','PP','GF','GC','DIF'])
df

,Equipos,PTS,PJ,PG,PE,PP,GF,GC,DIF
0,Universidad Católica,65,34,18,11,5,65,35,+30
1,Unión La Calera,57,34,17,6,11,59,41,+18
2,Universidad de Chile,52,34,13,13,8,49,33,+16
3,Unión Española,52,34,14,10,10,55,53,+2
4,Palestino,51,34,14,9,11,49,45,+4
5,Deportes Antofagasta,48,34,12,12,10,43,42,+1
6,Cobresal,47,34,13,8,13,45,40,+5
7,Huachipato,46,34,13,7,14,43,44,−1
8,Curicó Unido,46,34,13,7,14,40,52,−12
9,O'Higgins,45,34,12,9,13,40,39,+1
